In [2]:
from sklearn.pipeline import Pipeline
from sklearn import preprocessing
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer
from sklearn.neighbors import LocalOutlierFactor
from sklearn.base import TransformerMixin
from sklearn.preprocessing import MinMaxScaler,StandardScaler,RobustScaler
from sklearn.preprocessing import OneHotEncoder
from catboost import CatBoostRegressor
import pandas as pd
import numpy as np


In [32]:
iris = pd.read_csv('https://raw.githubusercontent.com/mwaskom/seaborn-data/master/iris.csv')
iris.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


,sepal_length,sepal_width,petal_length,petal_width
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2
...,...,...,...,...
145,6.7,3.0,5.2,2.3
146,6.3,2.5,5.0,1.9
147,6.5,3.0,5.2,2.0
148,6.2,3.4,5.4,2.3


In [34]:
x = iris.drop(columns=['species'])
y = iris['species']

In [63]:
class DFSimpleImputer(TransformerMixin):
    def fit_transform(self,x,y):
        print('fit/trans')
        self.fit(x,y)
        return self.transform(x)
    def transform(self, x):
        print('trans')
        print(x.shape)
        return x
    def fit(self,x,y):
        print('fit')
        return x

In [64]:
from sklearn.ensemble import RandomForestClassifier
pipe = Pipeline(steps=[
    ('impute_num', DFSimpleImputer()),
    ('model',RandomForestClassifier())
    ])

In [ ]:
from sklearn import set_config
set_config(display='diagram')   
# diplays HTML representation in a jupyter context
column_trans  

In [65]:
pipe.fit(x,y)

fit/trans
fit
trans
(150, 4)


Pipeline(memory=None,
         steps=[('impute_num',
                 <__main__.DFSimpleImputer object at 0x000001E3D4769860>),
                ('model',
                 RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                        class_weight=None, criterion='gini',
                                        max_depth=None, max_features='auto',
                                        max_leaf_nodes=None, max_samples=None,
                                        min_impurity_decrease=0.0,
                                        min_impurity_split=None,
                                        min_samples_leaf=1, min_samples_split=2,
                                        min_weight_fraction_leaf=0.0,
                                        n_estimators=100, n_jobs=None,
                                        oob_score=False, random_state=None,
                                        verbose=0, warm_start=False))],
         verbose=False)

In [66]:
from sklearn.model_selection import cross_validate
cross_validate(pipe, x,y,cv=3)

fit/trans
fit
trans
(100, 4)
trans
(50, 4)
fit/trans
fit
trans
(100, 4)
trans
(50, 4)
fit/trans
fit
trans
(100, 4)
trans
(50, 4)


{'fit_time': array([0.46672773, 0.59866142, 0.60865641]),
 'score_time': array([0.04297948, 0.03597403, 0.03497887]),
 'test_score': array([0.98, 0.94, 0.98])}

In [42]:
pipe.fit(x,y)

fit/trans


Pipeline(memory=None,
         steps=[('impute_num',
                 <__main__.DFSimpleImputer object at 0x000001E3D36A1D68>),
                ('model',
                 RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                        class_weight=None, criterion='gini',
                                        max_depth=None, max_features='auto',
                                        max_leaf_nodes=None, max_samples=None,
                                        min_impurity_decrease=0.0,
                                        min_impurity_split=None,
                                        min_samples_leaf=1, min_samples_split=2,
                                        min_weight_fraction_leaf=0.0,
                                        n_estimators=100, n_jobs=None,
                                        oob_score=False, random_state=None,
                                        verbose=0, warm_start=False))],
         verbose=False)

In [2]:
import pandas as pd

In [5]:
import numpy as np
df= pd.DataFrame({'col1':[1,2,3,4,np.nan,np.nan],'col2':[0,0,0,100,np.nan,np.nan]})
df

,col1,col2
0,1.0,0.0
1,2.0,0.0
2,3.0,0.0
3,4.0,100.0
4,NaN,NaN
5,NaN,NaN


In [94]:
from sklearn.base import TransformerMixin,BaseEstimator
from sklearn.impute import SimpleImputer
class DFZeroOrMeanImputer(TransformerMixin): # only for numeric features
    def fit(self, X, y=None):
        def determine_zero(series):
            perc_zero =  sum(series==0) / series.count()
            if perc_zero > 0.05: #zero occurs often: 
                return True
            else: 
                return False
        
        self.zero_cols = set([c for c in X.columns if determine_zero(X[c])])
        self.other_cols = set(X.columns) - self.zero_cols
        
        if self.zero_cols:
            self.zero_imp = SimpleImputer(strategy='constant', fill_value=0)
            self.zero_imp.fit(X[self.zero_cols])

        if self.other_cols:
            self.other_imp = SimpleImputer(strategy='mean')
            self.other_imp.fit(X[self.other_cols])
        return self
    
    def transform(self, X,y=None):
        zero_trans, other_trans = pd.DataFrame(),pd.DataFrame(),
        if self.zero_cols: zero_trans = pd.DataFrame(self.zero_imp.transform(X[self.zero_cols]), columns = self.zero_cols)
        if self.other_cols: other_trans = pd.DataFrame(self.other_imp.transform(X[self.other_cols]), columns = self.other_cols)
        return pd.concat([other_trans,zero_trans], axis=1)

In [95]:
zom = DFZeroOrMeanImputer()
zom.fit(df)
zom.transform(df)

,col1,col2
0,1.0,0.0
1,2.0,0.0
2,3.0,0.0
3,4.0,100.0
4,2.5,0.0
5,2.5,0.0


In [89]:
df['col2'].count()

4

In [272]:

for i, imp_val in enumerate(imp.specific_imps['MeadowV'].statistics_):
    print(i,imp_val,imp_val==imp_val)

0 160 True
1 RM True
2 21.0 True
3 1533 True
4 2 True
5 nan False
6 4 True
7 Lvl True
8 Inside True
9 3 True
10 MeadowV True
11 Norm True
12 Norm True
13 TwnhsE True
14 2Story True
15 4 True
16 6 True
17 1970 True
18 1970 True
19 Gable True
20 CompShg True
21 CemntBd True
22 CmentBd True
23 None True
24 0.0 True
25 3 True
26 3 True
27 CBlock True
28 4.0 True
29 3.0 True
30 0.0 True
31 6.0 True
32 0 True
33 1.0 True
34 0 True
35 0 True
36 630 True
37 GasA True
38 3 True
39 Y True
40 SBrkr True
41 630 True
42 0 True
43 0 True
44 630 True
45 1 True
46 0 True
47 1 True
48 0 True
49 3 True
50 1 True
51 3 True
52 6 True
53 8 True
54 0 True
55 3.0 True
56 Attchd True
57 1970.0 True
58 Unf True
59 1 True
60 0 True
61 3.0 True
62 3.0 True
63 2 True
64 0 True
65 0 True
66 0 True
67 0 True
68 0 True
69 0 True
70 nan False
71 MnPrv True
72 nan False
73 0 True
74 5 True
75 2009 True
76 WD True
77 Normal True
78 2 True
79 2 True
80 nan False
81 1.0 True
82 1.0 True
83 1.0 True
84 4 True
85 1 True
86

In [270]:
[imp.overall_imp[i] if pd.isna(imp_val) else imp_val for i, imp_val in enumerate(imp.specific_imps['MeadowV'].statistics_)]

TypeError: 'SimpleImputer' object is not subscriptable

In [276]:
imp.overall_imp

SimpleImputer(add_indicator=False, copy=True, fill_value=None,
              missing_values=nan, strategy='most_frequent', verbose=0)

In [264]:
imp.specific_imps['MeadowV'].statistics_[70] == imp.specific_imps['MeadowV'].statistics_[70]

False

In [257]:
import math 
math.isnan(imp.specific_imps['MeadowV'].statistics_[70])

True

In [265]:
pd.isna(imp.specific_imps['MeadowV'].statistics_[70])

True

In [266]:
pd.isna('la')

False

In [290]:
train_x[cat_x].infer_objects().dtypes

MSZoning         category
LandContour      category
LotConfig        category
Neighborhood     category
Condition1       category
Condition2       category
BldgType         category
HouseStyle       category
RoofStyle        category
RoofMatl         category
Exterior1st      category
Exterior2nd      category
MasVnrType       category
Foundation       category
Heating          category
CentralAir       category
Electrical       category
GarageType       category
GarageFinish     category
Fence            category
MiscFeature      category
SaleType         category
SaleCondition    category
dtype: object

In [279]:
aoc.nan_inspect(a)

0    1456
dtype: int64


ValueError: cannot concatenate unaligned mixed dimensional NDFrame objects

In [292]:
train_x.drop(columns=['Neighborhood']).shape

(1456, 176)

In [298]:
from sklearn.compose import make_column_selector
m = make_column_selector(dtype_exclude=np.number)

In [334]:
class test():
    def __init__(self,dtype_exclude):
        self.dtype_exclude = dtype_exclude
    def __call__(self,train):
        return make_column_selector(dtype_exclude = self.dtype_exclude)(train)+['Neighborhood']

In [335]:
from sklearn.compose import make_column_selector
a = test(dtype_exclude=np.number)

In [336]:
b = a(train_x)

In [98]:
def bla(**kwargs):
    print(kwargs)
bla(hi=2,isl=3)

{'hi': 2, 'isl': 3}
